In [1]:
import mysql.connector

try:
    # Establish a connection to the MySQL server
    cnx = mysql.connector.connect(user='root', password='',
                                host='127.0.0.1', database='eegsignal')
    cursor = cnx.cursor()

    # Define the table name and column names
    table_name = 'signals'
    columns = ['id', 'subset', 'state'] + [f'channel_{i}' for i in range(100)]

    # Define the data types for each column
    column_types = ['INT PRIMARY KEY AUTO_INCREMENT', 'VARCHAR(1)', 'VARCHAR(20)'] + ['INTEGER' for i in range(100)]

    # Construct the CREATE TABLE statement
    create_table = f"CREATE TABLE {table_name} ({', '.join([f'{column} {column_type}' for column, column_type in zip(columns, column_types)])})"

    # Execute the CREATE TABLE statement
    cursor.execute(create_table)

    # Commit the changes and close the connection
    cnx.commit()
    cnx.close()

except Exception as e:
    print(e)


1050 (42S01): Table 'signals' already exists


In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import os

# setting up spark
# spark = SparkSession.builder \
#     .appName("TD") \
#     .getOrCreate()

spark = SparkSession.builder.config('spark.port.maxRetries', 100).getOrCreate()

# path where the data is store
PATH = "./data/"

# mapping of the different state for the different subset
subset_category = {
    "A": "healthy",
    "B": "healthy",
    "C": "interictal",
    "D": "interictal",
    "E": "ictal"
}

# database configuration
user = 'root'
password = '' 
host = '127.0.0.1' 
database = 'eegsignal' 
table = 'signals'

# columns = ['id', 'subset', 'state'] + [f'channel_{i}' for i in range(100)]
columns = [f'channel_{i}' for i in range(100)]

import mysql.connector

try:
    # Establish a connection to the MySQL server
    cnx = mysql.connector.connect(user=user, password=password,
                                host=host, database=database)

    if cnx.is_connected():
        cursor = cnx.cursor()
        print('Connected to MySQL database')

except Exception as e:
    print(e)


for root, folders, files in os.walk(PATH):
    for file in files:
        data_path = os.path.join(PATH, file)
        # get the subset
        subset = file.split("_")[1].split(".")[0]
        # get the state
        state = subset_category[subset]

        print(f"\nPreparing data for subset {subset}")

        # load the data to be added in database
        data_signal = spark.read.csv(data_path, header=True)

        # change the name of the channel columns
        for i in range(len(data_signal.columns)):
            data_signal = data_signal.withColumnRenamed(data_signal.columns[i], columns[i])
        
        # add new columns
        data_signal = data_signal.withColumn("subset", lit(subset))
        data_signal = data_signal.withColumn("state", lit(state))

        cols = ", ".join(data_signal.columns)
        vals = ("%s," * 101) + "%s"
        
        print(f"Inserting data for subset {subset}")

        # prepare the insert statement with placeholders
        query = f"INSERT INTO {table} ({cols}) VALUES ({vals})"

        # convert the dataframe to a list of tuples
        data = [tuple(x) for x in data_signal.collect()] 

        for d in data:
            # execute the bulk insert statement
            cursor.execute(query, d)

            # commit the changes to the database
            cnx.commit()

        
# close the cursor and connection
cursor.close()
cnx.close()


Py4JError: An error occurred while calling None.org.apache.spark.sql.SparkSession. Trace:
py4j.Py4JException: Constructor org.apache.spark.sql.SparkSession([class org.apache.spark.SparkContext, class java.util.HashMap]) does not exist
	at py4j.reflection.ReflectionEngine.getConstructor(ReflectionEngine.java:179)
	at py4j.reflection.ReflectionEngine.getConstructor(ReflectionEngine.java:196)
	at py4j.Gateway.invoke(Gateway.java:237)
	at py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
	at py4j.commands.ConstructorCommand.execute(ConstructorCommand.java:69)
	at py4j.ClientServerConnection.waitForCommands(ClientServerConnection.java:182)
	at py4j.ClientServerConnection.run(ClientServerConnection.java:106)
	at java.base/java.lang.Thread.run(Thread.java:829)

